In [3]:
# Set up graph-tool environment
!conda install -y -c conda-forge graph-tool --quiet 

Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - graph-tool


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _x86_64-microarch-level-4  |           3_zen4          10 KB  conda-forge
    at-spi2-atk-2.38.0         |       h0630a04_3         332 KB  conda-forge
    at-spi2-core-2.40.3        |       h0630a04_0         643 KB  conda-forge
    atk-1.0-2.38.0             |       h04ea711_2         348 KB  conda-forge
    aws-c-auth-0.9.1           |       h48c9088_3         120 KB  conda-forge
    aws-c-cal-0.9.2            |       he7b75e1_1          50 KB  conda-forge
    aws-c-common-0.12.4        |       hb03c661_0         231 KB  conda-forge
    aws-c-compression-0.3.1    |       h92c474e_6          22 KB  conda-forge
    aws-

In [1]:
import os
os.getcwd()


'/home/jovyan/diss-essay-3/Code'

In [2]:
## Set up packages, so that my notebook runs inside an enviornment where graph-tool is saved and operating

import pandas as pd 
import graph_tool.all as gt
import matplotlib.pyplot as plt
import numpy as np

print("graph-tool imported successfully!")

graph-tool imported successfully!


In [3]:
### Construct the June 2015 network ("professionally managed" only); using the same format as the looped files

np.random.seed(1234) # Set a seed to be used consistently between the different years' network constructions

sorted_062015 = pd.read_csv('../Data/Cleaned files/NRED HOA - 06 2015_subset.csv') 

sorted_062015

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County,Address2_Normalized,Firm_Normalized,Phone_Normalized,Firm_ByPhone,Firm_ByAddress,Firm_Canonical,Firm_Final
0,Bella Terra @ Southern Highlands,SAME AS MASTER,SAM,705,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
1,Brasado I & Ii @ Southern Highlands,SAME AS MASTER,SAM,224,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
2,Castellina Ii @ Southern Highlands,SAME AS MASTER,SAM,30,NaN,Olympia Management Services,11411 Southern Highlands Ste 300,Las Vegas,NV,89141,Clark,11411 southern highland,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
3,Cortona Hills @ Southern Highlands,SAME AS MASTER,SAM,119,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
4,Glenhaven @ Southern Highlands,SAME AS MASTER,SAM,116,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,"Croix Home Owners' Association, The",0492912006-3,REG,137,(805) 962-0400,"La Playa Henderson, Lp",25 E Anapamu Street Ste 3,Santa Barbara,CA,90401,NaN,25 e anapamu st,la playa henderson lp,8.059620e+09,la playa henderson lp,la playa henderson lp,la playa henderson lp,la playa henderson lp
2685,"Espania Homeowner'S Association, Inc",0584712006-0,REG,416,818-901-8800,Espania,14039 Sherman Way Ste 201,Van Nuys,CA,91605,Clark,14039 sherman way,espania,8.189019e+09,espania,espania,espania,espania
2686,Rancho Del Sol Homeowners' Association,0370062006-3,REG,80,(858) 864-8196,Rancho Del Sol Llc,560 N Coast Highway 101 Ste 1,Encinitas,CA,92024,Clark,560 n coast highway 101,rancho del sol,8.588648e+09,rancho del sol,rancho del sol,rancho del sol,rancho del sol
2687,Lot 68 Tahoe Village Unit 1 Homeowners Associa...,1518-1979,SUB,4,(925) 323-1100,Tahoe Village Homeowners Association,2481 French Oak Place,Livermore,CA,94550,Douglas,2481 french oak place,tahoe village homeowner association,9.253231e+09,tahoe village homeowner association,tahoe village homeowner association,tahoe village homeowner association,tahoe village homeowner association


In [4]:
## Visualizations

unique_companies_062015 = sorted_062015['Firm_Final'].unique()  # Pull out the individual management company names, changed to final
total_nv_network_062015 = gt.Graph()  # Create an empty graph to populate with the data

# Any decisions regarding which managed ones to keep in (e.g. individually registered) and how to label by size/type?

hoa_vertices_062015 = {} # Set empty dictionaries with the respective nodes
company_vertices_062015 = {}

company_name_prop_062015 = total_nv_network_062015.new_vertex_property("string")
company_city_prop_062015 = total_nv_network_062015.new_vertex_property("string")
company_state_prop_062015 = total_nv_network_062015.new_vertex_property("string") 

for company_062015 in unique_companies_062015: # Sort through the management companies, store their information, and add as vertices to the graph
    company_data_062015 = sorted_062015[sorted_062015['Firm_Final'] == company_062015]
    company_vertex = total_nv_network_062015.add_vertex()
    company_vertices_062015[company_062015] = company_vertex
    company_name_prop_062015[company_vertex] = company_062015

    city = company_data_062015.iloc[0]['City']  
    state = company_data_062015.iloc[0]['State']  
    company_city_prop_062015[company_vertex] = city  
    company_state_prop_062015[company_vertex] = state 
    
    for index, row in company_data_062015.iterrows(): # Iterate through the company data, add the unique HOAs (which are now rows in the transformed data), and draw edges between the node types
        hoa_name = row['Name']
        if hoa_name not in hoa_vertices_062015:
            hoa_vertex = total_nv_network_062015.add_vertex()
            hoa_vertices_062015[hoa_name] = hoa_vertex
        else:
            hoa_vertex = hoa_vertices_062015[hoa_name]
        total_nv_network_062015.add_edge(hoa_vertex, company_vertex)

pos = gt.sfdp_layout(total_nv_network_062015, K=120, C=0.01) # Space out the nodes using the force-directed algorithm

# Color and sizing
vertex_color = total_nv_network_062015.new_vertex_property("vector<double>") # Assign color scheme based on HOA or management (and latter's location)

for v in total_nv_network_062015.vertices():
    if v in company_vertices_062015.values():
        city = company_city_prop_062015[v]
        state = company_state_prop_062015[v]
        if state != "NV":
            vertex_color[v] = (0.5, 0, 0.5, 0.75)  # Purple color for companies with address outside NV
        elif city in ["Boulder City", "Henderson", "Las Vegas", "Mesquite", "North Las Vegas", "Blue Diamond", "Bunkerville", "Cal-Nev-Ari", "Enterprise", "Goodsprings", "Indian Springs", "Laughlin", "Moapa Town", "Moapa Valley", "Mount Charleston", "Nelson", "Paradise", "Sandy Valley", "Searchlight", "Spring Valley", "Summerlin South", "Sunrise Manor", "Whitney", "Winchester"]:
            vertex_color[v] = (0, 1, 0, 0.75)  # Green color for Vegas MSA-headquartered companies
        else:
            vertex_color[v] = (1, 0, 0, 0.75)  # Red color for other cities in NV
    else:
        vertex_color[v] = (0, 0, 1, 0.75)  # Blue color for all individual HOAs, inferring membership
        
# Graphic creation and save
gt.graph_draw(total_nv_network_062015, pos, vertex_fill_color=vertex_color, output_size=(2000, 2000), output="../Output/062015_managed.png") # Draw and save the graphic to the project folder

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fb419bfcc90, at 0x7fb419c13e90>

In [5]:
## Statistics (for professionally managed HOAs)

# Degree centrality: add up total number of edges (representing company-HOA connections), sort by the most, and calculate proportion
degrees = total_nv_network_062015.degree_property_map('total')
total_degree_centrality_sum = total_nv_network_062015.num_edges()
top_10 = sorted(company_vertices_062015.values(), key=lambda v: degrees[v], reverse=True)[:10]
print("Top 10 Vertices by Degree Centrality - June 2015:")
for i, vertex in enumerate(top_10):
    company_name = company_name_prop_062015[vertex]
    degree_centrality = degrees[vertex]
    proportion_of_total = (degree_centrality / total_degree_centrality_sum)*100
    print(f"{i+1}. {company_name}, Number of HOAs: {degree_centrality}, Proportion of Total: {proportion_of_total:.1f}")

# Average community size by firm (in units): group by company, add up number of units responsible for and number of HOAs, and sort by the most
company_stats_062015 = sorted_062015.groupby('Firm_Final').agg({'# of Units': 'sum', 'Name': 'count'})
company_stats_062015['Average Community Size'] = company_stats_062015['# of Units'] / company_stats_062015['Name']
company_stats_062015['Average Community Size'] = company_stats_062015['Average Community Size'].round()
top_10_average_community_size = company_stats_062015.sort_values(by='Average Community Size', ascending=False).head(10)
print("Top 10 Companies by Average Community Size - June 2015:")
print(top_10_average_community_size)

Top 10 Vertices by Degree Centrality - June 2015:
1. firstservice residential nevada, Number of HOAs: 339, Proportion of Total: 12.6
2. camco, Number of HOAs: 225, Proportion of Total: 8.4
3. mp association management, Number of HOAs: 166, Proportion of Total: 6.2
4. terra west management service, Number of HOAs: 165, Proportion of Total: 6.1
5. colonial property management, Number of HOAs: 115, Proportion of Total: 4.3
6. nicklin property management, Number of HOAs: 114, Proportion of Total: 4.2
7. associa nevada south, Number of HOAs: 93, Proportion of Total: 3.5
8. community management group, Number of HOAs: 86, Proportion of Total: 3.2
9. thoroughbred management, Number of HOAs: 82, Proportion of Total: 3.0
10. ideal community management, Number of HOAs: 67, Proportion of Total: 2.5
Top 10 Companies by Average Community Size - June 2015:
                                           # of Units  Name  \
Firm_Final                                                    
coyote spring invest

In [6]:
### Construct the June 2015 network (all)
### Should appear the same but with freestanding yellow nodes for non-managed (under my definition)

In [8]:
### Construct the June 2025 network ("professionally managed" only)

# Necessary packages are already loaded in

np.random.seed(1234)

sorted_062025 = pd.read_csv('../Data/Cleaned files/NRED HOA - 06 2025_subset.csv')  
sorted_062025

,Name,SOS#,Assn Type,# of Units,Telephone,Address1,Address2,City,State,Zip Code,County,Address2_Normalized,Firm_Normalized,Phone_Normalized,Firm_ByPhone,Firm_ByAddress,Firm_Canonical,Firm_Final
0,Alpian Meadow @ Mountain'S Edge,SAME AS MASTER,SAM,84.0,NaN,Capital Consultants Management Corporation,8015 Blue Diamond #120,Las Vegas,NV,89178,Clark,8015 blue diamond 120,capital consultant management corporation,NaN,olympia management service,capital consultant management corporation,olympia management service,olympia management service
1,Aspire @ Mountain'S Edge,SAME AS MASTER,SAM,59.0,NaN,Capital Consultants Management Corporation,8015 Blue Diamond #120,Las Vegas,NV,89178,Clark,8015 blue diamond 120,capital consultant management corporation,NaN,olympia management service,capital consultant management corporation,olympia management service,olympia management service
2,Bella Terra @ Southern Highlands,SAME AS MASTER,SAM,705.0,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
3,Brasado I & Ii @ Southern Highlands,SAME AS MASTER,SAM,224.0,NaN,Olympia Management Services,11411 Southern Highlands Pkwy Ste 120,Las Vegas,NV,89141,Clark,11411 southern highland pkwy,olympia management service,NaN,olympia management service,olympia management service,olympia management service,olympia management service
4,Carriage Club @ Green Valley Ranch,SAME AS MASTER,SAM,96.0,NaN,Terra West Management Services,Po Box 80900,Las Vegas,NV,89180,Clark,po box 80900,terra west management service,NaN,olympia management service,terra west management service,olympia management service,olympia management service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3455,Cassia Community Association,13813982021-4,REG,207.0,(949) 855-1800,Seabreeze Management Company,8329 W Sunset Ste 270,Las Vegas,NV,89113,Clark,8329 w sunset,seabreeze management company,9.498552e+09,seabreeze management company,seabreeze management company,seabreeze management company,seabreeze management company
3456,Deer Springs Homeowners Association,0360862014-7,REG,89.0,(949) 855-1800,Seabreeze Management,8329 W Sunset Rd Suite 270,Las Vegas,NV,89113,Clark,8329 w sunset rd,seabreeze management,9.498552e+09,seabreeze management company,seabreeze management company,seabreeze management company,seabreeze management company
3457,Mateo Community Association,0164112015-1,REG,126.0,(949) 855-1800,Seabeeze Management Company,8329 W Sunset Rd Ste 270,Las Vegas,NV,89113,Clark,8329 w sunset rd,seabeeze management company,9.498552e+09,seabreeze management company,seabreeze management company,seabreeze management company,seabreeze management company
3458,Spinnaker Homes At Centennial Hills Homeowners...,28248-2002,REG,298.0,(949) 855-1800,Seabreeze Management Company,8329 W Sunset Rd Ste 270,Las Vegas,NV,89113,Clark,8329 w sunset rd,seabreeze management company,9.498552e+09,seabreeze management company,seabreeze management company,seabreeze management company,seabreeze management company


In [9]:
## Visualizations

unique_companies_062025 = sorted_062025['Firm_Final'].unique()  
total_nv_network_062025 = gt.Graph()  

hoa_vertices_062025 = {} 
company_vertices_062025 = {}

# Any decisions regarding which managed ones to keep in (e.g. individually registered) and how to label by size/type?

company_name_prop_062025 = total_nv_network_062025.new_vertex_property("string")
company_city_prop_062025 = total_nv_network_062025.new_vertex_property("string")
company_state_prop_062025 = total_nv_network_062025.new_vertex_property("string") 

for company_062025 in unique_companies_062025: 
    company_data_062025 = sorted_062025[sorted_062025['Firm_Final'] == company_062025]
    company_vertex = total_nv_network_062025.add_vertex()
    company_vertices_062025[company_062025] = company_vertex
    company_name_prop_062025[company_vertex] = company_062025

    city = company_data_062025.iloc[0]['City']  
    state = company_data_062025.iloc[0]['State']  
    company_city_prop_062025[company_vertex] = city  
    company_state_prop_062025[company_vertex] = state 
    
    for index, row in company_data_062025.iterrows(): 
        hoa_name = row['Name']
        if hoa_name not in hoa_vertices_062025:
            hoa_vertex = total_nv_network_062025.add_vertex()
            hoa_vertices_062025[hoa_name] = hoa_vertex
        else:
            hoa_vertex = hoa_vertices_062025[hoa_name]
        total_nv_network_062025.add_edge(hoa_vertex, company_vertex)

pos = gt.sfdp_layout(total_nv_network_062025, K=50) 

# Color and sizing
vertex_color = total_nv_network_062025.new_vertex_property("vector<double>") 

for v in total_nv_network_062025.vertices():
    if v in company_vertices_062025.values():
        city = company_city_prop_062025[v]
        state = company_state_prop_062025[v]
        if state != "NV":
            vertex_color[v] = (0.5, 0, 0.5, 0.75)  # Purple color for companies with address outside NV
        elif city in ["Boulder City", "Henderson", "Las Vegas", "Mesquite", "North Las Vegas", "Blue Diamond", "Bunkerville", "Cal-Nev-Ari", "Enterprise", "Goodsprings", "Indian Springs", "Laughlin", "Moapa Town", "Moapa Valley", "Mount Charleston", "Nelson", "Paradise", "Sandy Valley", "Searchlight", "Spring Valley", "Summerlin South", "Sunrise Manor", "Whitney", "Winchester"]:
            vertex_color[v] = (0, 1, 0, 0.75)  # Green color for Vegas MSA-headquartered companies
        else:
            vertex_color[v] = (1, 0, 0, 0.75)  # Red color for other cities in NV
    else:
        vertex_color[v] = (0, 0, 1, 0.75)  # Blue color for all individual HOAs
        
# Graphic creation and save
gt.graph_draw(total_nv_network_062025, pos, vertex_fill_color=vertex_color, output_size=(1000, 1000), output="../Output/062025_managed.png") # Draw and save the graphic to the project folder

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fb418efe9d0, at 0x7fb41ced0f50>

In [11]:
## Statistics (for professionally managed HOAs)

# Degree centrality
degrees = total_nv_network_062025.degree_property_map('total')
total_degree_centrality_sum = total_nv_network_062025.num_edges()
top_10 = sorted(company_vertices_062025.values(), key=lambda v: degrees[v], reverse=True)[:10]
print("Top 10 Vertices by Degree Centrality - June 2025:")
for i, vertex in enumerate(top_10):
    company_name = company_name_prop_062025[vertex]
    degree_centrality = degrees[vertex]
    proportion_of_total = (degree_centrality / total_degree_centrality_sum)*100
    print(f"{i+1}. {company_name}, Number of HOAs: {degree_centrality}, Proportion of Total: {proportion_of_total:.1f}")

# Average community size by firm 
company_stats_062025 = sorted_062025.groupby('Firm_Final').agg({'# of Units': 'sum', 'Name': 'count'})
company_stats_062025['Average Community Size'] = company_stats_062025['# of Units'] / company_stats_062025['Name']
company_stats_062025['Average Community Size'] = company_stats_062025['Average Community Size'].round()
top_10_average_community_size = company_stats_062025.sort_values(by='Average Community Size', ascending=False).head(10)
print("Top 10 Companies by Average Community Size - June 2025:")
print(top_10_average_community_size)

Top 10 Vertices by Degree Centrality - June 2025:
1. firstservice residential, Number of HOAs: 291, Proportion of Total: 8.4
2. camco, Number of HOAs: 216, Proportion of Total: 6.2
3. terra west management service, Number of HOAs: 198, Proportion of Total: 5.7
4. nevada community management, Number of HOAs: 164, Proportion of Total: 4.7
5. colonial community management, Number of HOAs: 159, Proportion of Total: 4.6
6. the management trust, Number of HOAs: 147, Proportion of Total: 4.2
7. taylor association management, Number of HOAs: 133, Proportion of Total: 3.8
8. las vega community management group, Number of HOAs: 128, Proportion of Total: 3.7
9. thoroughbred management, Number of HOAs: 127, Proportion of Total: 3.7
10. prime community management, Number of HOAs: 120, Proportion of Total: 3.5
Top 10 Companies by Average Community Size - June 2025:
                                           # of Units  Name  \
Firm_Final                                                    
coyote spr

In [ ]:
### Construct the June 2025 network (all)

In [ ]:
## Rewiring (going to need to adjust to perform in parallel, as well as use an adjacency matrix)

# Step 1: Log transform the actual structure's degree values
logged_degrees_actual = np.log(total_nv_network_062025.degree_property_map('total').a)

# Step 2: Build the random rewiring function (only changing edges from HOAs to other possible companies, not each other), iterate and check progress, and fill in list of results
def rewire_edges(graph):
    edges = list(graph.edges())
    np.random.shuffle(edges)
    for edge in edges:
        source, target = edge
        source_degree = graph.degree_property_map('total')[source]
        target_degree = graph.degree_property_map('total')[target]
        if source_degree == 1 and target_degree > 1:
            possible_targets = [v for v in graph.vertices() if v!= source and graph.degree_property_map('total')[v] > 1 and not graph.edge(source, v)]
            new_target = np.random.choice(possible_targets)
            graph.remove_edge(edge)
            graph.add_edge(source, new_target)
    return graph

iterations = 100 # Can play around with different amounts
rewired_networks = []
for i in range(iterations):
    print(f"Iteration {i+1}/{iterations}")
    rewired_network = rewire_edges(total_nv_network_062025.copy())
    rewired_networks.append(rewired_network)
    
# Step 3: Log transform the rewirings' degree values and calculate the mean and SD of them sample population we have created
logged_degrees_rewired = []
for rewired_network in rewired_networks:
    degrees_rewired = rewired_network.degree_property_map('total').a
    logged_degrees_rewired.append(np.log(degrees_rewired))

mean_logged_degrees_rewired = np.mean(logged_degrees_rewired, axis=0)
std_logged_degrees_rewired = np.mean(logged_degrees_rewired, axis=0)

# Step 4: Calculate the z-scores for the rewirings using above information
z_scores = []
for degrees in logged_degrees_rewired:
    z_score = (degrees - mean_logged_degrees_rewired) / std_logged_degrees_rewired
    z_scores.append(z_score)

# Step 5: Calculate the actual structure's z-scores using above information
actual_z_score = (logged_degrees_actual - - mean_logged_degrees_rewired) / std_logged_degrees_rewired

# Step 6: Plot the results

In [ ]:
# Looping and tracking clusters across entire time period (refer to the g-t package dictionary for clues)